In [1]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import os
import re
from shapely.geometry import Polygon, MultiPolygon
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from rasterstats import zonal_stats
from osgeo import gdal
import rasterio
import rasterstats
import time
sys.path.append(os.path.abspath("D:/other_thesis/clean_sheet/codes/functions/"))
import functions_model #import python file containing the functions

In [2]:
# For updates
import importlib
importlib.reload(functions_model)

<module 'functions_model' from 'D:\\other_thesis\\clean_sheet\\codes\\functions\\functions_model.py'>

In [3]:
# Import files related to yields and insurance units
yields = gpd.read_file(r'D:\other_thesis\codes_latestversion\general\files\yields_geometry_largestarea.geojson')
district_yield = pd.read_excel('D:/Data_download/Other/yield_district/district_yields.xlsx')

## PADDY

In [4]:
# Create dataframes per season per year such that we can calculate statistics based on this information (e.g. parameter value differs per year, season)
yields_paddy_2016_kharif_gp = yields[(yields['Year'] == 2016) & (yields['Season'] == 'Kharif') & (yields['Crop'] == 'paddy') & (yields['Gram Panchayat/Hobli'] == 'gp')] # 2306
yields_paddy_2017_kharif_gp = yields[(yields['Year'] == 2017) & (yields['Season'] == 'Kharif') & (yields['Crop'] == 'paddy') & (yields['Gram Panchayat/Hobli'] == 'gp')] # 1956
yields_paddy_2018_kharif_gp = yields[(yields['Year'] == 2018) & (yields['Season'] == 'Kharif') & (yields['Crop'] == 'paddy') & (yields['Gram Panchayat/Hobli'] == 'gp')] # 2078

In [5]:
yields_paddy_2016_kharif_gp.head(1)

,Year,Season,Insurance Unit,shapeName,Gram Panchayat/Hobli,District,Taluk,Crop,IRR_RF,Average Yield(Kg/Ha),geometry
160,2016,Kharif,amarapura,amarapura,gp,ballari,ballari,paddy,irr,6157.78,"POLYGON ((77.04634 15.15551, 77.04628 15.15518..."


In [6]:
7115-(2306+1956+2078)

775

In [7]:
# unique_paddy_2016kgp = yields_paddy_2016_kharif_gp.drop_duplicates(subset = ['Year', 'Season', 'Insurance Unit', 'shapeName', 'Gram Panchayat/Hobli', 'District', 'Taluk', 'Crop', 'geometry']) # 2134
# unique_paddy_2017kgp = yields_paddy_2017_kharif_gp.drop_duplicates(subset = ['Year', 'Season', 'Insurance Unit', 'shapeName', 'Gram Panchayat/Hobli', 'District', 'Taluk', 'Crop', 'geometry']) # 1792
# unique_paddy_2018kgp = yields_paddy_2018_kharif_gp.drop_duplicates(subset = ['Year', 'Season', 'Insurance Unit', 'shapeName', 'Gram Panchayat/Hobli', 'District', 'Taluk', 'Crop', 'geometry']) # 1909

# PADDY UNTOUCHED

In [8]:
# This cell sets values for the parameters we use for function time_observations within the next cell
paths = ['D:/other_thesis/clean_sheet/Data/monthly_images2/NDVI/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/EVI/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/FAPAR/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/LAI/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/SIF/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/SSM/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/SUSM/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/LSTd/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/LSTn/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/PCPN/']
methods = [False, False, False, False, True, True, True, False, False, True]
statistic = ['median', 'mean', 'percentile_10', 'percentile_90', 'min', 'max']
parameters = ['NDVI', 'EVI', 'FAPAR', 'LAI', 'SIF', 'SSM', 'SUSM', 'LSTd', 'LSTn', 'PCPN']

In [9]:
df_paddy_2016k = yields_paddy_2016_kharif_gp.copy()
df_paddy_2017k = yields_paddy_2017_kharif_gp.copy()
df_paddy_2018k = yields_paddy_2018_kharif_gp.copy()
for i in range(len(parameters)):
       df_paddy_2016k = functions_model.time_observations(df_paddy_2016k, parameters[i], paths[i], statistic, methods[i])
       df_paddy_2017k = functions_model.time_observations(df_paddy_2017k, parameters[i], paths[i], statistic, methods[i])
       df_paddy_2018k = functions_model.time_observations(df_paddy_2018k, parameters[i], paths[i], statistic, methods[i])
       print('Parameter ' + parameters[i] + ' finished.')
paddy_kharif = pd.concat([df_paddy_2016k, df_paddy_2017k, df_paddy_2018k], ignore_index=True)
# Time: 25 min (maize), 40 min (both),  min (paddy)

Parameter NDVI finished.
Parameter EVI finished.
Parameter FAPAR finished.
Parameter LAI finished.
Parameter SIF finished.
Parameter SSM finished.
Parameter SUSM finished.
Parameter LSTd finished.
Parameter LSTn finished.
Parameter PCPN finished.


In [10]:
paddy_kharif = functions_model.soiltype_one(paddy_kharif, 'Soiltype', 'D:/Data_download/Other/Soiltype/soiltype_Karnataka.tif', ['majority'], False) # Add soiltype majority for each observation

In [11]:
# Add the district yield(kg/ha) of the previous year to the dataframe
paddy_kharif['Previous District Yield(Kg/Ha)'] = np.nan
for i in paddy_kharif.index:
    paddy_kharif.loc[i,'Previous District Yield(Kg/Ha)'] = float(district_yield[(district_yield['District'] == paddy_kharif.loc[i,'District']) & (district_yield['Crop'] == paddy_kharif.loc[i,'Crop']) & (district_yield['Year'] == paddy_kharif.loc[i,'Year'] - 1) & (district_yield['Season'] == paddy_kharif.loc[i,'Season'])]['Yield(Kg/Ha)'])

In [12]:
paddy_kharif.to_file(r'D:\other_thesis\clean_sheet\files\df_pre\paddy_kharif_params_untouched_large2.geojson', driver="GeoJSON")# 6337, 371

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


# PADDY TOUCHED

In [13]:
# This cell sets values for the parameters we use for function time_observations within the next cell
paths = ['D:/other_thesis/clean_sheet/Data/monthly_images2/NDVI/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/EVI/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/FAPAR/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/LAI/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/SIF/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/SSM/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/SUSM/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/LSTd/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/LSTn/', 'D:/other_thesis/clean_sheet/Data/monthly_images2/PCPN/']
methods = [False, False, False, False, True, True, True, True, True, True]
statistic = ['median', 'mean', 'percentile_10', 'percentile_90', 'min', 'max']
parameters = ['NDVI', 'EVI', 'FAPAR', 'LAI', 'SIF', 'SSM', 'SUSM', 'LSTd', 'LSTn', 'PCPN']

In [14]:
df_paddy_2016k_t = yields_paddy_2016_kharif_gp.copy()
df_paddy_2017k_t = yields_paddy_2017_kharif_gp.copy()
df_paddy_2018k_t = yields_paddy_2018_kharif_gp.copy()
for i in range(len(parameters)):
       df_paddy_2016k_t = functions_model.time_observations(df_paddy_2016k_t, parameters[i], paths[i], statistic, methods[i])
       df_paddy_2017k_t = functions_model.time_observations(df_paddy_2017k_t, parameters[i], paths[i], statistic, methods[i])
       df_paddy_2018k_t = functions_model.time_observations(df_paddy_2018k_t, parameters[i], paths[i], statistic, methods[i])
       print('Parameter ' + parameters[i] + ' finished.')
paddy_kharif_t = pd.concat([df_paddy_2016k_t, df_paddy_2017k_t, df_paddy_2018k_t], ignore_index=True)
# Time: 25 min (maize), 40 min (both),  min (paddy)

Parameter NDVI finished.
Parameter EVI finished.
Parameter FAPAR finished.
Parameter LAI finished.
Parameter SIF finished.
Parameter SSM finished.
Parameter SUSM finished.
Parameter LSTd finished.
Parameter LSTn finished.
Parameter PCPN finished.


In [15]:
paddy_kharif_t = functions_model.soiltype_one(paddy_kharif_t, 'Soiltype', 'D:/Data_download/Other/Soiltype/soiltype_Karnataka.tif', ['majority'], False) # Add soiltype majority for each observation

In [16]:
# Add the district yield(kg/ha) of the previous year to the dataframe
paddy_kharif_t['Previous District Yield(Kg/Ha)'] = np.nan
for i in paddy_kharif_t.index:
    paddy_kharif_t.loc[i,'Previous District Yield(Kg/Ha)'] = float(district_yield[(district_yield['District'] == paddy_kharif_t.loc[i,'District']) & (district_yield['Crop'] == paddy_kharif_t.loc[i,'Crop']) & (district_yield['Year'] == paddy_kharif_t.loc[i,'Year'] - 1) & (district_yield['Season'] == paddy_kharif_t.loc[i,'Season'])]['Yield(Kg/Ha)'])

In [17]:
paddy_kharif_t.to_file(r'D:\other_thesis\clean_sheet\files\df_pre\paddy_kharif_params_touched_large2.geojson', driver="GeoJSON")# 6337, 371

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
